In [1]:
import streamlit as st
import PyPDF2
from langchain.chains import StuffDocumentsChain,MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv

load_dotenv()
log_data = [] 
import sys
sys.path.append('../..')

azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2024-02-15-preview"
llm_name = "gpt-4o"


In [6]:


from langchain_community.document_loaders import ConfluenceLoader
confluence_username = os.getenv("CONFLUENCE_USERNAME")
confluence_password = os.getenv("CONFLUENCE_PASSWORD")


loader = ConfluenceLoader(
    url="https://wiki.softtech.com.tr/", username=confluence_username, api_key=confluence_password
)
documents1= loader.load(space_key="YPZ", include_attachments=False, limit=100, max_pages = 1000, include_restricted_content = False)
print(len(documents1))

loader = ConfluenceLoader(
    url="https://wiki.softtech.com.tr/", username=confluence_username, api_key=confluence_password
)
documents2 = loader.load(space_key="YPZ", include_attachments=False, limit=100, max_pages = 1000, include_restricted_content = True)
print(len(documents2))

[print(doc.metadata) for doc in documents1]
[print(doc.metadata) for doc in documents2]









Received runtime arguments {'space_key': 'YPZ', 'include_attachments': False, 'limit': 100, 'max_pages': 1000, 'include_restricted_content': False}. Passing runtime args to `load` is deprecated. Please pass arguments during initialization instead.
Received runtime arguments {'space_key': 'YPZ', 'include_attachments': False, 'limit': 100, 'max_pages': 1000, 'include_restricted_content': True}. Passing runtime args to `load` is deprecated. Please pass arguments during initialization instead.


74
75
{'title': 'AI Lab', 'id': '86148988', 'source': 'https://wiki.softtech.com.tr/display/YPZ/AI+Lab', 'when': '2024-08-13T11:40:34.243+03:00'}
{'title': 'frAIday', 'id': '89752943', 'source': 'https://wiki.softtech.com.tr/display/YPZ/frAIday', 'when': '2024-06-06T09:37:28.877+03:00'}
{'title': '2023-05-12 Haftalık Statü ve Paylaşım', 'id': '89753887', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89753887', 'when': '2023-05-30T19:51:48.068+03:00'}
{'title': 'Toplantı Kayıtları', 'id': '89754463', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89754463', 'when': '2023-05-23T11:02:22.118+03:00'}
{'title': 'Google AI süreçlerinin deneyimlenmesi-1', 'id': '89754465', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89754465', 'when': '2023-05-30T19:42:41.547+03:00'}
{'title': '2023-05-26 Haftalık Statü ve Paylaşım', 'id': '89754714', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89754714', 'when': '2

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [18]:
for doc in split_docs:
    print(doc.metadata)

{'title': 'AI Lab', 'id': '86148988', 'source': 'https://wiki.softtech.com.tr/display/YPZ/AI+Lab', 'when': '2024-08-13T11:40:34.243+03:00'}
{'title': 'frAIday', 'id': '89752943', 'source': 'https://wiki.softtech.com.tr/display/YPZ/frAIday', 'when': '2024-06-06T09:37:28.877+03:00'}
{'title': '2023-05-12 Haftalık Statü ve Paylaşım', 'id': '89753887', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89753887', 'when': '2023-05-30T19:51:48.068+03:00'}
{'title': 'Google AI süreçlerinin deneyimlenmesi-1', 'id': '89754465', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89754465', 'when': '2023-05-30T19:42:41.547+03:00'}
{'title': '2023-05-26 Haftalık Statü ve Paylaşım', 'id': '89754714', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89754714', 'when': '2023-05-30T19:56:13.811+03:00'}
{'title': '2023-06-02 Haftalık Statü ve Paylaşım', 'id': '89755436', 'source': 'https://wiki.softtech.com.tr/pages/viewpage.action?pageId=89755436

In [6]:
# First, we specify the LLMChain to use for mapping each document to an individual summary:
llm=AzureChatOpenAI(model="gpt-4o", temperature=0, api_key=azure_oai_key, azure_endpoint=azure_oai_endpoint, api_version=api_version)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer in the language of the provided documents:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)


# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer in the language of the provided documents::"""
reduce_prompt = PromptTemplate.from_template(reduce_template)


# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)


# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

result = map_reduce_chain.invoke(split_docs)

result["output_text"]

'Bu belgelerden yola çıkarak ana temaları şu şekilde özetleyebiliriz:\n\n1. **Yatırım Stratejileri ve Sihirli Formül**:\n   - Joel\'in düşük değerlemelerle iyi işletmelerin hisse senetlerini içeren ve "sihirli formül" olarak adlandırılan basit, mantıklı ve düşük riskli yatırım yöntemi tanıtılmaktadır. Bu formülün uzun vadede yüksek getiri sağladığı ve piyasa üstünlüğü elde edilebileceği belirtilmektedir.\n   - Yatırım stratejilerinin uzun vadede mantıklı ve zamansız olması gerektiği, geçmiş performansın geleceği belirlememesi ve profesyonel tahminlerin zorluğu vurgulanmaktadır.\n   - Yatırım sürecinin en az üç ila beş yıl boyunca kararlılıkla sürdürülmesi gerektiği ifade edilmektedir.\n\n2. **Portföy Yönetimi ve Çeşitlendirme**:\n   - 20-30 hisselik bir portföy oluşturmanın risk dağılımı açısından önemi ve az sayıda iyi şirket seçiminin önerilmesi.\n   - Portföyün düzenli aralıklarla hisse senedi alım ve satım işlemleriyle yönetilmesi ve her bir hisse senedinin bir yıl tutulduktan sonr

In [ ]:
def main():

    st.set_page_config(page_title="PDF Summarizer")
    st.title("PDF Summarazing App")
    st.write("Summarize your pdf files in just a few seconds.")
    # Displaying a description
    st.divider() # Inserting a divider for better layout
    # Creating a file uploader widget to upload PDF files
    pdf = st. file_uploader ('Upload your PDF Document', type = "pdf")
    # Creating a button for users to submit their PDF

    submit = st.button("Generate Summary")
    if submit and pdf is not None:
        summary = process_uploaded_files(pdf)
        st.subheader("Summary of the file:")
        st.write(summary)


if __name__ == '__main__':
    main()